In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import subprocess
import time
from random import sample
import ipfshttpclient
from os import listdir
from os.path import isfile, join
import re
import praw

import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

from substrate_helpers import *
from database_queries import update_db
import sqlite3

/home/chia/miniconda3/lib/python3.8/site-packages/ipfshttpclient/client/__init__.py:73: VersionMismatch: Unsupported daemon version '0.13.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


In [4]:
accounts = json.load(open("accounts.json", "r"))
schemas = json.load(open("schemas.json", "r"))

In [5]:
reddit_creds = json.load(open(".reddit_creds.json", "r"))
reddit = praw.Reddit(
    client_id=reddit_creds["client_id"],
    client_secret=reddit_creds["client_secret"],
    password=reddit_creds["password"],
    user_agent=reddit_creds["user_agent"],
    username=reddit_creds["username"],
)

In [6]:
substrate = SubstrateInterface(
    url="ws://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)
bob = Keypair.create_from_uri('//Bob')
bob_msa_id = get_msa_id(bob)

In [7]:
con = sqlite3.connect('postthreadV1_write.db')
cur = con.cursor()

In [8]:
r_all = reddit.subreddit('all')

In [9]:
client = ipfshttpclient.connect()

In [10]:
path = "/tmp/"

In [11]:
def mint_reddit_users_msa_ids(post, delegate):
    ## USER ##
    try:
        username = str(post.author.name)
        if username[0:2] == '0x':
            username = username[2:]
        profile_pic = post.author.icon_img
    except:
        username = "removed"
        profile_pic = "removed"

    password = 'password'
    user_wallet = Keypair.create_from_uri('//' + username + password)
    create_msa_with_delegator(delegate, user_wallet, wait_for_inclusion=False, wait_for_finalization=False)
    
    return {"username": username, "password": password, "profile_pic": profile_pic, "user_wallet": user_wallet}

In [12]:
def mint_reddit_users_msa_ids_for_posts(posts, delegate):
    users = {}
    posts_and_comments = posts
    for i, (post_name, post) in enumerate(posts.items()):
        users[post_name] = {post_name: mint_reddit_users_msa_ids(post, delegate)}
        
        top_comments = []
        for comment in post.comments.list()[:10]:
            if comment.parent_id == post.name:
                top_comments.append(comment.name)
                users[post_name][comment.name] = mint_reddit_users_msa_ids(post, delegate)
        for comment in post.comments.list():
            if type(comment) != praw.models.reddit.more.MoreComments and comment.parent_id in top_comments:
                users[post_name][comment.name] = mint_reddit_users_msa_ids(post, delegate)
        
    return users

In [13]:
def mint_comment(post_data_hash, parent_hash, user, comment):      
    user_wallet = Keypair.create_from_uri('//' + user['username'] + user['password'])
    user_msa_id = get_msa_id(user_wallet)

    comment_data = {
        "post_hash": post_data_hash,
        "parent_hash": parent_hash,
        "depth": comment.depth,
        "body": comment.body,
    }

    comment_data_hash, receipt_comment = mint_data(comment_data, user_msa_id, schemas['comment'], path+'comments/', wait_for_inclusion=False, wait_for_finalization=False)

    ## comment votes ##
    receipt_ups = mint_votes(user_msa_id, comment.ups, comment_data_hash, post_data_hash, 'comment')
    receipt_downs = mint_votes(user_msa_id, comment.downs, comment_data_hash, post_data_hash, 'comment')
    
    return comment_data_hash, user_msa_id

In [14]:
def mint_reddit_posts_and_users(posts, users, delegate):
    for i, (post_name, post) in enumerate(posts.items()):
        user = users[post.name][post.name]
        ## POST ##
        post_data = {
            "category": post.subreddit.display_name,
            "title": post.title,
            "body": post.selftext,
            "url": post.url,
            "is_nsfw": post.over_18
        }
        user_wallet = Keypair.create_from_uri('//' + user['username'] + user['password'])
        user_msa_id = get_msa_id(user_wallet)
        users[post.name][post.name]['user_msa_id'] = user_msa_id
        if user_msa_id is None:
            print('no user_msa_id')
            continue
        
        receipt_user = mint_user(user_msa_id, user['username'], user['password'], user['profile_pic'], user_wallet)

        post_data_hash, receipt_post = mint_data(post_data, user_msa_id, schemas['post'], path+'posts/', 
                                                 wait_for_inclusion=False, wait_for_finalization=False)

        ## post votes ##
        receipt_ups = mint_votes(user_msa_id, post.ups, post_data_hash, post_data_hash, 'post', 
                                                 wait_for_inclusion=False, wait_for_finalization=False)
        receipt_downs = mint_votes(user_msa_id, post.downs, post_data_hash, post_data_hash, 'post', 
                                                 wait_for_inclusion=False, wait_for_finalization=False)
        
        comment_list = post.comments.list()
        top_comments = {}
        for comment in comment_list[:10]:
            if comment.parent_id == post.name:  
                try:
                    user = users[post.name][comment.name]     
                except:
                    continue
                comment_data_hash, comment_user_msa_id = mint_comment(post_data_hash, post_data_hash, user, comment)
                users[post.name][comment.name]['user_msa_id'] = comment_user_msa_id
                top_comments[comment.name] = {"hash": comment_data_hash, "comment_count": 0}
                
        if len(comment_list) > 10:
            for comment in comment_list[10:]:
                if type(comment) != praw.models.reddit.more.MoreComments and comment.parent_id in top_comments and top_comments[comment.parent_id]['comment_count'] < 10:
                    try:    
                        user = users[post.name][comment.name]    
                    except:
                        continue
                        
                    comment_data_hash = mint_comment(post_data_hash, top_comments[comment.parent_id]["hash"], user, comment)
                    users[post.name][comment.name]['user_msa_id'] = comment_user_msa_id
                    top_comments[comment.name] = {"hash": comment_data_hash, "comment_count": 0}
                    top_comments[comment.parent_id]["comment_count"] += 1
        
        ## FOLLOWS ##
        for comment_name in sample(top_comments.keys(), min(10, len(top_comments))):
            follow_user(user_msa_id, users[post_name][comment_name]['user_msa_id'])
            
        for comment_name in sample(top_comments.keys(), min(100, len(top_comments))):
            follow_user(users[post_name][comment_name]['user_msa_id'], user_msa_id)
        
        
    return True

# DB writer

In [15]:
all_users = {}
all_posts = {}

In [ ]:
minted_time = 0
last_block = 0
while True:
    if (time.time() - minted_time) / 60 > 60:
        posts = {p.name: p for p in r_all.top(time_filter='hour') if p.name not in all_posts}
        all_posts.update(posts)
        all_users.update(mint_reddit_users_msa_ids_for_posts(posts, bob))
        time.sleep(60*1)
        print("waiting for msa's to post..")
        mint_reddit_posts_and_users(posts, all_users, bob)

        time_to_mint_posts = False
        minted_time = time.time()
        print('Done minting')
    
    current_block = substrate.get_block()['header']['number']
    if current_block != last_block:
        last_block = update_db(
            query_start=True, 
            backfill=False, schemaToUpdate=None
        )
        print("Block: ", current_block)
        
    time.sleep(1)

waiting for msa's to post..
Done minting
post 58
comment 806
vote 1726
user 58
follow 1270
Block:  8105
post 15
comment 140
vote 312
user 15
follow 250
Block:  8106
post 17
comment 142
vote 318
user 17
follow 219
Block:  8107
post 4
comment 26
vote 60
user 4
follow 67
Block:  8108
Block:  8109
Block:  8110
Block:  8111
Block:  8112
Block:  8113
Block:  8114
Block:  8115
Block:  8116
Block:  8117
Block:  8118
Block:  8119
Block:  8120
Block:  8121
Block:  8122
Block:  8123
Block:  8124
Block:  8125
Block:  8126
Block:  8127
Block:  8128
comment 1
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
follow 1
Block:  8129
comment 1
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 

comment 1
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Block:  8161
comment 1
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Block:  8162
comment 2
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice Post!', 'is_nsfw': False}
Block:  8163
comment 2
Failed to get data from row  {'post_hash': 'QmaPVDDt

comment 3
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice Post!', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmcM1M1tDFPtBY1EfEFXXGms6uuiMPq8nbJsg7smi4GNGX', 'parent_comment_hash': 'QmcM1M1tDFPtBY1EfEFXXGms6uuiMPq8nbJsg7smi4GNGX', 'depth': 0, 'body': 'That looks awesome', 'is_nsfw': False}
Block:  8177
comment 3
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY

comment 3
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice Post!', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmcM1M1tDFPtBY1EfEFXXGms6uuiMPq8nbJsg7smi4GNGX', 'parent_comment_hash': 'QmcM1M1tDFPtBY1EfEFXXGms6uuiMPq8nbJsg7smi4GNGX', 'depth': 0, 'body': 'That looks awesome', 'is_nsfw': False}
Block:  8189
comment 3
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY

comment 3
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice Post!', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmcM1M1tDFPtBY1EfEFXXGms6uuiMPq8nbJsg7smi4GNGX', 'parent_comment_hash': 'QmcM1M1tDFPtBY1EfEFXXGms6uuiMPq8nbJsg7smi4GNGX', 'depth': 0, 'body': 'That looks awesome', 'is_nsfw': False}
Block:  8201
comment 3
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'parent_comment_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY9', 'depth': 0, 'body': 'Nice post', 'is_nsfw': False}
Failed to get data from row  {'post_hash': 'QmaPVDDt24iqiSBuPAvPYY86LmyZrfKEw33ceLP81mFXY

...

Block:  15649
Block:  15650
Block:  15651
Block:  15652
Block:  15653
Block:  15654
Block:  15655
Block:  15656
Block:  15657
Block:  15658
Block:  15659
Block:  15660
Block:  15661
Block:  15662
Block:  15663
Block:  15664
Block:  15665
Block:  15666
Block:  15667
Block:  15668
Block:  15669
Block:  15670
Block:  15671
Block:  15672
Block:  15673
Block:  15674
Block:  15675
Block:  15676
Block:  15677
Block:  15678
Block:  15679
Block:  15680
Block:  15681
Block:  15682
Block:  15683
Block:  15684
Block:  15685
Block:  15686
Block:  15687
Block:  15688
Block:  15689
Block:  15690
Block:  15691
Block:  15692
Block:  15693
Block:  15694
Block:  15695
Block:  15696
Block:  15697
Block:  15698
Block:  15699
Block:  15700
Block:  15701
Block:  15702
Block:  15703
Block:  15704
Block:  15705
Block:  15706
Block:  15707
Block:  15708
Block:  15709
Block:  15710
Block:  15711
Block:  15712
Block:  15713
Block:  15714
Block:  15715
waiting for msa's to post..
Done minting
post 63
comment 588
v

Block:  16252
Block:  16253
Block:  16254
Block:  16255
Block:  16256
Block:  16257
Block:  16258
Block:  16259
Block:  16260
Block:  16261
Block:  16262
Block:  16263
Block:  16264
Block:  16265
Block:  16266
Block:  16267
Block:  16268
Block:  16269
Block:  16270
Block:  16271
Block:  16272
Block:  16273
Block:  16274
Block:  16275
Block:  16276
Block:  16277
Block:  16278
Block:  16279
Block:  16280
Block:  16281
Block:  16282
Block:  16283
Block:  16284
Block:  16285
Block:  16286
Block:  16287
Block:  16288
Block:  16289
Block:  16290
Block:  16291
Block:  16292
Block:  16293
Block:  16294
Block:  16295
Block:  16296
Block:  16297
Block:  16298
Block:  16299
Block:  16300
Block:  16301
Block:  16302
Block:  16303
Block:  16304
Block:  16305
Block:  16306
Block:  16307
Block:  16308
Block:  16309
Block:  16310
Block:  16311
Block:  16312
Block:  16313
Block:  16314
Block:  16315
Block:  16316
Block:  16317
Block:  16318
Block:  16319
Block:  16320
Block:  16321
Block:  16322
Block:

Block:  16851
Block:  16852
Block:  16853
Block:  16854
Block:  16855
Block:  16856
Block:  16857
Block:  16858
Block:  16859
Block:  16860
Block:  16861
Block:  16862
Block:  16863
Block:  16864
Block:  16865
Block:  16866
Block:  16867
Block:  16868
Block:  16869
Block:  16870
Block:  16871
Block:  16872
Block:  16873
Block:  16874
Block:  16875
Block:  16876
Block:  16877
Block:  16878
Block:  16879
Block:  16880
Block:  16881
Block:  16882
Block:  16883
Block:  16884
Block:  16885
Block:  16886
Block:  16887
Block:  16888
Block:  16889
Block:  16890
Block:  16891
Block:  16892
Block:  16893
Block:  16894
Block:  16895
Block:  16896
Block:  16897
Block:  16898
waiting for msa's to post..
Done minting
post 69
comment 624
vote 1386
user 69
follow 1056
Block:  16914
post 20
comment 70
vote 180
user 20
follow 130
Block:  16915
post 11
comment 75
vote 172
user 11
follow 143
Block:  16916
Block:  16917
Block:  16918
Block:  16919
Block:  16920
Block:  16921
Block:  16922
Block:  16923
Blo

Block:  17428
Block:  17429
Block:  17430
Block:  17431
Block:  17432
Block:  17433
Block:  17434
Block:  17435
Block:  17436
Block:  17437
Block:  17438
Block:  17439
Block:  17440
Block:  17441
Block:  17442
Block:  17443
Block:  17444
Block:  17445
Block:  17446
Block:  17447
Block:  17448
Block:  17449
Block:  17450
Block:  17451
Block:  17452
Block:  17453
Block:  17454
Block:  17455
Block:  17456
Block:  17457
Block:  17458
Block:  17459
Block:  17460
Block:  17461
Block:  17462
Block:  17463
Block:  17464
Block:  17465
Block:  17466
Block:  17467
Block:  17468
Block:  17469
Block:  17470
Block:  17471
Block:  17472
Block:  17473
Block:  17474
Block:  17475
Block:  17476
Block:  17477
Block:  17478
Block:  17479
Block:  17480
Block:  17481
Block:  17482
Block:  17483
Block:  17484
Block:  17485
Block:  17486
Block:  17487
Block:  17488
Block:  17489
Block:  17490
Block:  17491
Block:  17492
Block:  17493
Block:  17494
Block:  17495
Block:  17496
Block:  17497
Block:  17498
Block:

Block:  18006
Block:  18007
Block:  18008
Block:  18009
Block:  18010
Block:  18011
Block:  18012
Block:  18013
Block:  18014
Block:  18015
Block:  18016
Block:  18017
Block:  18018
Block:  18019
Block:  18020
Block:  18021
Block:  18022
Block:  18023
Block:  18024
Block:  18025
Block:  18026
Block:  18027
Block:  18028
Block:  18029
Block:  18030
Block:  18031
Block:  18032
Block:  18033
Block:  18034
Block:  18035
Block:  18036
Block:  18037
Block:  18038
Block:  18039
Block:  18040
Block:  18041
Block:  18042
Block:  18043
Block:  18044
waiting for msa's to post..
Done minting
post 67
comment 464
vote 1062
user 67
follow 822
Block:  18058
post 18
comment 73
vote 182
user 18
follow 122
Block:  18059
post 15
comment 52
vote 134
user 15
follow 124
Block:  18060
Block:  18061
Block:  18062
Block:  18063
Block:  18064
Block:  18065
Block:  18066
Block:  18067
Block:  18068
Block:  18069
Block:  18070
Block:  18071
Block:  18072
Block:  18073
Block:  18074
Block:  18075
Block:  18076
Bloc

In [33]:
update_db(backfill=True, schemaToUpdate="link")

post
Skipping post
comment
Skipping comment
vote
Skipping vote
user
Skipping user
follow
Skipping follow
link
link 2
Failed to parse json {"account_type": gmail,"account_value": "example@gmail.com"}
payout
Skipping payout


15790

In [84]:
update_db(backfill=True, schemaToUpdate=None)

post 10000
Failed to get ipfs hash  {"nothing": nothing}
Failed to get data from row  {'post_hash': 'QmUY2PmpYNSnon8RBzwJhxd8DCXMH1PCbhnFMu5TgZskPP', 'parent_comment_hash': 'string', 'depth': 0, 'body': 'test comment', 'is_nsfw': True}
Failed to get data from row  {'post_hash': 'QmUY2PmpYNSnon8RBzwJhxd8DCXMH1PCbhnFMu5TgZskPP', 'parent_comment_hash': 'string', 'depth': 0, 'body': 'test comment', 'is_nsfw': True}
Failed to get data from row  {'post_hash': 'QmUY2PmpYNSnon8RBzwJhxd8DCXMH1PCbhnFMu5TgZskPP', 'parent_comment_hash': 'string', 'depth': 0, 'body': 'a comment', 'is_nsfw': True}
Failed to get data from row  {'post_hash': 'QmUY2PmpYNSnon8RBzwJhxd8DCXMH1PCbhnFMu5TgZskPP', 'parent_comment_hash': 'string', 'depth': 0, 'body': 'a comment', 'is_nsfw': True}
comment 10000
vote 10000
user 10000
follow 10000


15281

In [294]:
pd.read_sql_query("""    SELECT * FROM vote
    LIMIT 0, 10""", con)

,post_hash,parent_hash,parent_type,num_votes,block_number,msa_id_from_query,provider_key,date_minted
0,QmW4gfvWVdCw4fy4powQhK7c9cnDvGY6oGF1W4idfhkPXy,QmW4gfvWVdCw4fy4powQhK7c9cnDvGY6oGF1W4idfhkPXy,post,118538,177,7,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:09:42'
1,QmW4gfvWVdCw4fy4powQhK7c9cnDvGY6oGF1W4idfhkPXy,QmW4gfvWVdCw4fy4powQhK7c9cnDvGY6oGF1W4idfhkPXy,post,0,178,7,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:10:00'
2,QmW4gfvWVdCw4fy4powQhK7c9cnDvGY6oGF1W4idfhkPXy,QmW4gfvWVdCw4fy4powQhK7c9cnDvGY6oGF1W4idfhkPXy,post,118538,181,7,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:11:18'
3,QmW4gfvWVdCw4fy4powQhK7c9cnDvGY6oGF1W4idfhkPXy,QmW4gfvWVdCw4fy4powQhK7c9cnDvGY6oGF1W4idfhkPXy,post,0,181,7,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:11:18'
4,QmcoKf7whNof9z1hKfDiYuHBAt1AEvD7fdiC5JWaAxBkF9,QmcoKf7whNof9z1hKfDiYuHBAt1AEvD7fdiC5JWaAxBkF9,post,91687,181,8,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:11:18'
5,QmcoKf7whNof9z1hKfDiYuHBAt1AEvD7fdiC5JWaAxBkF9,QmcoKf7whNof9z1hKfDiYuHBAt1AEvD7fdiC5JWaAxBkF9,post,0,181,8,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:11:18'
6,QmbL3efVgMgfQRCLQ7QiH2YSVjDqNxHgkvWP9Lde9cQNiJ,QmbL3efVgMgfQRCLQ7QiH2YSVjDqNxHgkvWP9Lde9cQNiJ,post,92313,181,9,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:11:18'
7,QmbL3efVgMgfQRCLQ7QiH2YSVjDqNxHgkvWP9Lde9cQNiJ,QmbL3efVgMgfQRCLQ7QiH2YSVjDqNxHgkvWP9Lde9cQNiJ,post,0,181,9,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:11:18'
8,QmTouht5ngCno1YkYfAijrTbR7PXmQggtrRoMsa21teC38,QmTouht5ngCno1YkYfAijrTbR7PXmQggtrRoMsa21teC38,post,76949,181,10,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:11:18'
9,QmTouht5ngCno1YkYfAijrTbR7PXmQggtrRoMsa21teC38,QmTouht5ngCno1YkYfAijrTbR7PXmQggtrRoMsa21teC38,post,0,181,10,5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty,'2022-06-23 16:11:18'


In [ ]:
pd.read_sql_query("""SELECT user.date_minted FROM user, post, comment, vote, follow limit 1""", con)

In [289]:
con = sqlite3.connect('postthreadV1_write.db')
cur = con.cursor()

In [298]:
set_connection(con, cur)

In [313]:
last_updated_block = update_db(
    start_block=0, 
    backfill=True, schemaToUpdate=None
)

post 103
comment 1003
vote 4670
user 326
follow 1988


In [272]:

    # copy write database to read for REST api to access
    bashCommand = "rm postthreadV1_read.db"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)

    new_db = sqlite3.connect('postthreadV1_read.db') # create a memory database

    query = "".join(line for line in con.iterdump())

    # Dump old database in the new one. 
    new_db.executescript(query)
    new_db.close()

OperationalError: table comment already exists

In [288]:
con.close()

# Old

In [20]:
def mint_comments(comments, post, post_data_hash):
    id_to_hash = {}
    for comment in comments:
        try:
            username = comment.author.name
            profile_pic = comment.author.icon_img
        except:
            username = "removed"
            profile_pic = "removed"
            
        password = 'password'
        user_wallet = Keypair.create_from_uri('//' + username + password)
        comment_user_msa_id, receipt_comment_user = mint_user(username, password, profile_pic, user_wallet)
            
        if comment.parent_id == post.name:
            parent_hash = post_data_hash
        else:
            parent_hash = id_to_hash[comment.parent_id]
            
        comment_data = {
            "post_hash": post_data_hash,
            "parent_hash": parent_hash,
            "depth": comment.depth,
            "body": comment.body,
        }
        
        comment_data_hash, receipt_comment = mint_data(comment_data, user_msa_id, path+'comments/' wait_for_inclusion=False, wait_for_finalization=False)
        
        receipt_comment = make_call("Messages", "add", call_params, bob, wait_for_inclusion=False, wait_for_finalization=False)
        print(comment_data_hash)
        
        ## comment votes ##
        receipt_ups = mint_votes(comment_user_msa_id, comment.ups, comment_data_hash, post_data_hash, 'comment')
        receipt_downs = mint_votes(comment_user_msa_id, comment.downs, comment_data_hash, post_data_hash, 'comment')
        print(comment.ups, comment.downs)
        
        ## FOLLOWS ##
        for follow_msa_id in sample(users, min(10, len(users))):
            follow_user(comment_user_msa_id, follow_msa_id)
            
        users.append(comment_user_msa_id)

In [74]:
c.children

['idfh90g',
 'idhlodd',
 'idfzo8y',
 'idgpexs',
 'idflc6e',
 'idgtpmq',
 'idg250u',
 'idfuefl',
 'idghn5g',
 'idhrhpx',
 'idg14ie',
 'idf549m',
 'idg4vrw']

In [23]:
while (True):
    posts = [p for p in r_all.top(time_filter='day')]
    for i, post in enumerate(posts):
        ## USER ##
        try:
            username = post.author.name
            profile_pic = post.author.icon_img
        except:
            username = "removed"
            profile_pic = "removed"
            
        password = 'password'
        user_wallet = Keypair.create_from_uri('//' + username + password)
        user_msa_id, receipt_user = mint_user(username, password, profile_pic, user_wallet)

        ## POST ##
        post_data = {
            "category": post.subreddit.display_name,
            "username": username,
            "profile_pic": profile_pic,
            "title": post.title,
            "body": post.selftext,
            "url": post.url,
            "is_nsfw": post.over_18
        }

        post_data_hash, receipt_post = make_post(post_data, user_msa_id, path+'posts/', wait_for_inclusion=False, wait_for_finalization=False)
        print(post_data_hash)

        ## post votes ##
        receipt_ups = mint_votes(user_msa_id, post.ups, post_data_hash, post_data_hash, 'post')
        receipt_downs = mint_votes(user_msa_id, post.downs, post_data_hash, post_data_hash, 'post')
        print(post.ups, post.downs)

        ## COMMENT ##
        mint_comments(post.comments.list()[:10], post, post_data_hash)
        
        ## FOLLOWS ##
        for follow_msa_id in sample(users, min(10, len(users))):
            follow_user(user_msa_id, follow_msa_id)
            
        users.append(user_msa_id)
        
    print("waiting...")
    print()
    # sleep for a bit and then check again
    time.sleep(60*10)

Extrinsic '0xb5cafbb6c313f9261676610e390287c69a7b5739da89ae3b9b74386b113f3b62' sent and included in block 'None'
Extrinsic '0xd002df048664209bfd609f391cb13630d6e06006e77df33c439828fa94cc6db0' sent and included in block 'None'
Extrinsic '0x81bbb6952a69cc25cd539f35c975d31e026e3967e272587811bd9080eaab3ddc' sent and included in block 'None'
Extrinsic '0x68a1a0f067c6c36b3436a3db68774160c18392c5e29dcbfacdccde8fb72bbf20' sent and included in block 'None'
Extrinsic '0xb2f171be69c428488f4f4cad95adca69e2e3c5d822099675be711574e3e2d492' sent and included in block 'None'
Extrinsic '0xb3005c43b597882b171a7f5002a1bdf25a0ac11f111b28e14d3a419859cfd72c' sent and included in block 'None'
Extrinsic '0x65152b49fea7e38f6d34395618bffd635521d17f7bd5a46cdff1ad7c3111db1a' sent and included in block 'None'
Extrinsic '0x45f43a311919562d99b43feee8b6262edc6624fc97145e68168220b77b58a8f8' sent and included in block 'None'
Extrinsic '0x239052065565e66de58466c9b57cee27683e3c98fbf70220e41b0a4e1d94f1f8' sent and included

TypeError: 'NoneType' object is not subscriptable

In [39]:
b = reddit.redditor('abowlofspaghetti')

In [40]:
b.link_karma, b.comment_karma

(7003, 737, 0, 0)

In [47]:
b.submissions.new()

In [49]:
for t in b.submissions.new():
    break

In [52]:
t.selftext

"I'm not a huge fan of GBTC for Bitcoin but it's the only way to get exposure to crypto in my 401k. So I will be dumping all my GBTC whenever an ETF comes available. However, I feel like many people could do that and it'll tank the stock. Is that a possibility? I know it's supposed to represent btc value but it's constantly off by huge percentages."

In [32]:
[t for t in b.awardee_karma.top()]

[]

In [543]:
substrate.get_block()['header']['number']

9694

In [489]:
start_block = 5490

In [42]:
post_pattern = re.compile(f".*,.*,type,parent,data")

In [43]:
schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

post_schemas = {}
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    if post_pattern.match(schemaTemp.value):
        print(schemaTemp.value)
        post_schemas[schemaTemp.value] = i

QmQzgLfEczTXa3rWFC4y7vJMECV7yFqbByhFPzUR9woUWs,interestingasfuck,type,parent,data


In [48]:
%%time
content_jsons = {}
for schema, schemaId in post_schemas.items():
    params = [
        schemaId,
        {
            "page_size": 10000,
            "from_block": 0,
            "to_block": 10000,
            "from_index": 0,
        }
    ]

    content = substrate.rpc_request(
        method='messages_getBySchema',
        params=params,
    )
    print(schema, content)
    if len(content['result']['content']) > 0:
        content_jsons[schema] = content['result']['content']

QmQzgLfEczTXa3rWFC4y7vJMECV7yFqbByhFPzUR9woUWs,interestingasfuck,type,parent,data {'jsonrpc': '2.0', 'result': {'content': [{'block_number': 34, 'data': '0x2c2c706f73742c2c516d517a674c6645637a5458613372574643347937764a4d454356377946716242796846507a555239776f555773', 'index': 0, 'msa_id': 1, 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'}, {'block_number': 43, 'data': '0x2c2c706f73742c2c516d517a674c6645637a5458613372574643347937764a4d454356377946716242796846507a555239776f555773', 'index': 0, 'msa_id': 1, 'signer': '5FHneW46xGXgs5mUiveU4sbTyGBzmstUspZC92UhjJM694ty'}], 'has_next': False}, 'id': 139}
CPU times: user 2.16 ms, sys: 0 ns, total: 2.16 ms
Wall time: 41.8 ms


In [45]:
content_jsons

{}

In [23]:
posts

{'t3_vim7ng': Submission(id='vim7ng'),
 't3_vit3x1': Submission(id='vit3x1'),
 't3_vixi09': Submission(id='vixi09'),
 't3_viiej5': Submission(id='viiej5'),
 't3_vizh71': Submission(id='vizh71'),
 't3_vivnm4': Submission(id='vivnm4'),
 't3_viulb0': Submission(id='viulb0'),
 't3_vinjcr': Submission(id='vinjcr'),
 't3_vip13v': Submission(id='vip13v'),
 't3_vii7b8': Submission(id='vii7b8'),
 't3_virxw5': Submission(id='virxw5'),
 't3_vimk8e': Submission(id='vimk8e'),
 't3_vit8uu': Submission(id='vit8uu'),
 't3_viu2uy': Submission(id='viu2uy'),
 't3_viiwew': Submission(id='viiwew'),
 't3_vilwa7': Submission(id='vilwa7'),
 't3_viqm5t': Submission(id='viqm5t'),
 't3_vii5pa': Submission(id='vii5pa'),
 't3_vivotx': Submission(id='vivotx'),
 't3_vitu44': Submission(id='vitu44'),
 't3_vipp6u': Submission(id='vipp6u'),
 't3_viu8fu': Submission(id='viu8fu'),
 't3_viv1te': Submission(id='viv1te'),
 't3_vj1k3n': Submission(id='vj1k3n'),
 't3_vir64e': Submission(id='vir64e'),
 't3_viv9uf': Submission(

In [353]:
def add_comment(_data, _dict, _is_vote):
    for hsh in _dict.keys():
        if hsh == data[2]:
            if is_vote:
                if '-' in data[3]:
                    # downvote
                    _dict['data']['downvotes'] += data[3]
                else:
                    # upvote
                    _dict['data']['upvotes'] += data[3]
            else:
                _dict[data[3]] = {
                    'data': json.loads(client.cat(_data).decode()), 
                    'comments': {}
                }
                
                _dict['data'].update({"upvotes": 0, "downvotes": 0})
            return True
        result = add_comment(_data, _dict['comments'], _is_vote)
        if result:
            return True
    return False

In [ ]:
posts[0]

In [354]:
posts = {}
for schema, contents in content_jsons.items():
    schema_items = schema.split(',')
    category = schema_items[0]
    post_hash = schema_items[1]
    posts[post_hash] = {'data': {"upvotes": 0, "downvotes": 0}, 'comments': {}}
    
    for content in contents:
        data = bytes.fromhex(content['data'][2:]).decode().split(',')
        # votes had numeric numbers instead of hashes
        is_vote = data[3].strip('-').isnumeric() 
        if is_vote:
            data[3] = int(data[3])
            
        if data[2] == '': 
            # empty means its post data
            posts[post_hash]['data'].update(json.loads(client.cat(data[3]).decode()))
        elif data[2] == post_hash: 
            # post_hash means its comment
            if is_vote:
                if data[3] < 0:
                    # downvote
                    posts[post_hash]['data']['downvotes'] += data[3]
                else:
                    # upvote
                    posts[post_hash]['data']['upvotes'] += data[3]
            else:
                posts[post_hash]['comments'][data[3]] = {
                    'data': json.loads(client.cat(data[3]).decode()), 
                    'comments': {}
                }
                posts[post_hash]['comments'][data[3]]['data'].update({"upvotes": 0, "downvotes": 0})
        else: 
            # neither means its comment of comment
            result = add_comment(data[3], posts[post_hash]['comments'], is_vote)
            if not result:
                print('couldnt find', data[2], json.loads(client.cat(data[3]).decode()))

In [42]:
all_users['t3_vim7ng']

{'username': 'SamMee514',
 'password': 'password',
 'profile_pic': 'https://styles.redditmedia.com/t5_3nk7j/styles/profileIcon_xwxr5t82uvo71.jpg?width=256&height=256&crop=256:256,smart&s=f979360d063c5a7fac85612deb538dbba83f8fb4',
 'user_wallet': <Keypair (address=5HT8T1mywhQspN8ZwSVFh9g5o9BCnGWWuutWfddUTbAa7LNL)>}

In [355]:
from collections import OrderedDict
o = OrderedDict(sorted(posts.items(), key=lambda t: t[1]['data']['upvotes'], reverse=True))

In [41]:
all_users.keys(), all_posts.keys(), posts.keys()

(dict_keys(['t3_vim7ng', 't3_vit3x1', 't3_vixi09', 't3_viiej5', 't3_vizh71', 't3_vivnm4', 't3_viulb0', 't3_vinjcr', 't3_vip13v', 't3_vii7b8', 't3_virxw5', 't3_vimk8e', 't3_vit8uu', 't3_viu2uy', 't3_viiwew', 't3_vilwa7', 't3_viqm5t', 't3_vii5pa', 't3_vivotx', 't3_vitu44', 't3_vipp6u', 't3_viu8fu', 't3_viv1te', 't3_vj1k3n', 't3_vir64e', 't3_viv9uf', 't3_vitirc', 't3_virvdb', 't3_vixfko', 't3_vip47n', 't3_vin8hn', 't3_vik8zi', 't3_vit2m6', 't3_vis6co', 't3_vixs3e', 't3_vilfua', 't3_vis3q3', 't3_vij700', 't3_vj069m', 't3_vin9v0', 't3_viotqt', 't3_viltda', 't3_viwh64', 't3_viyqer', 't3_vitiqi', 't3_viq23j', 't3_vise9r', 't3_vil8qr', 't3_viilkv', 't3_viu2nn', 't3_viklwh', 't3_viqb3j', 't3_viyb5l', 't3_viz15t', 't3_vitrc6', 't3_vikfyz', 't3_vilc2a', 't3_vj2xbo', 't3_viko2m', 't3_vin4jv', 't3_viqtez', 't3_vilidd', 't3_vij5oj', 't3_vij6k3', 't3_viuxtj', 't3_vizthy', 't3_viquj1', 't3_viswlp', 't3_vii4wg', 't3_vir1eq', 't3_vij5j5', 't3_viv4dn', 't3_viv6b5', 't3_vis8mp', 't3_vipqbx', 't3_vispa0', 

In [357]:
posts_list = []
for k, v in OrderedDict(sorted(posts.items(), key=lambda t: t[1]['data']['upvotes'], reverse=True)).items():
    posts_list.append({k:v})